# Feature Engineering

In [1]:
import os
import numpy as np
import pandas as pd
import sklearn
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler,
    MinMaxScaler,
    PowerTransformer,
    FunctionTransformer
)
from feature_engine.encoding import (
	RareLabelEncoder,
    MeanEncoder,
    CountFrequencyEncoder
)
from feature_engine.datetime import DatetimeFeatures

In [2]:
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
sklearn.set_config(transform_output='pandas')

In [3]:
PROJECT_DIR = 'R:\Jaydeep/Flight-Price-Prediction'
DATA_DIR = 'data'

TRAIN_DATASET_NAME = 'train'
VAL_DATASET_NAME = 'validation'
TEST_DATASET_NAME  = 'test'

TRAIN_PRE_DATASET_NAME = 'train_preprocessing'
VAL_PRE_DATASET_NAME = 'val_preprocessing'
TEST_PRE_DATASET_NAME = 'test_preprocessing'

In [4]:
def get_dataset(dataset_name):
    file_name = f'{dataset_name}.csv'
    file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name)
    return pd.read_csv(file_path)

In [5]:
def export_dataset(X, name):
	file_name = f"{name}.csv"
	file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name)
	X.to_csv(file_path, index=False)

In [6]:
train_df = get_dataset(TRAIN_DATASET_NAME)

In [7]:
X_train = train_df.drop(columns=['price','additional_info'])
y_train = train_df.price.copy()

In [8]:
X_train.columns.to_list()

['airline',
 'date_of_journey',
 'source',
 'destination',
 'dep_time',
 'arrival_time',
 'duration_minute',
 'total_stops']

## Airline

In [9]:
airline_transformer = Pipeline(steps=[
    ('grouper', RareLabelEncoder(tol=0.1, replace_with='other', n_categories=2)),
    ('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

## Date of Journey

In [10]:
date_features = ['month', 'day_of_week', 'day_of_year']

In [11]:
date_tranformer = Pipeline(steps=[
    ('date_to_features', DatetimeFeatures(features_to_extract=date_features, yearfirst=True, format='mixed')),
    ('min_max_scaler', MinMaxScaler())
])

## Source & Destination

In [12]:
set(np.union1d(
    X_train.source.unique(),
    X_train.destination.unique()
))


{'Banglore',
 'Chennai',
 'Cochin',
 'Delhi',
 'Hyderabad',
 'Kolkata',
 'Mumbai',
 'New Delhi'}

In [13]:
location_df = X_train.loc[:, ['source', 'destination']]

In [14]:
location_transformer = Pipeline(steps=[
    ('grouper', RareLabelEncoder(tol=0.1, replace_with='other', n_categories=2)),
    ('mean_encoder', MeanEncoder()),
    ('power_transformer', PowerTransformer())
])

In [15]:
def is_north(X):
	columns = X.columns.to_list()
	north_cities = {"Delhi", "Kolkata", "Mumbai", "New Delhi"}
	return (
		X
		.assign(**{
			f"{col}_is_north": X.loc[:, col].isin(north_cities).astype(int)
			for col in columns
		})
		.drop(columns=columns)
	)

In [16]:
location_union_transformer = FeatureUnion(transformer_list=[
	("location_transformer", location_transformer),
	("is_north_transformer", FunctionTransformer(func=is_north))
])

## Departure Time & Arrival Time

In [17]:
time_df = X_train.loc[:, ['dep_time', 'arrival_time']]

In [18]:
time_transformer = Pipeline(
    steps = [
        ('dt', DatetimeFeatures(features_to_extract=['hour', 'minute'],  yearfirst=True, format='mixed')),
        ('scaler', MinMaxScaler())
    ]
)

In [19]:
def part_of_day(X, start=0 , mid=8, end=16):
    columns = X.columns.to_list()
    X_temp = X.assign(
        **{
            col: pd.to_datetime(X.loc[:, col]).dt.hour
            for col in columns
        }
    )
    return (
        X_temp
        .assign(
            **{
                f'{col}_part_of_day': np.select(
                    [
                        X_temp.loc[:, col].between(start, mid, inclusive='left'),
                        X_temp.loc[:, col].between(mid, end, inclusive='left')
                    ], choicelist = ['start', 'mid'] ,default = 'end'
                )
                for col in columns
            }
        ).drop(columns=columns)
    )

In [20]:
part_of_day_transformer = Pipeline(
    steps = [
        ('part_of_day_func', FunctionTransformer(func=part_of_day)),
        ('count_fre_encoder', CountFrequencyEncoder()),
        ('min_max_scaler', MinMaxScaler())
    ]
)

In [21]:
time_union_transformer = FeatureUnion(
    transformer_list=[
        ('time_transformer', time_transformer),
        ('part_of_day_transformer', part_of_day_transformer)
    ]
)

## Duration

In [22]:
duration_log_transformer = FunctionTransformer(func=np.log)

## Total Stop

In [23]:
def is_direct_flight(X):
    return X.assign(
        is_direct_flight = X.total_stops.eq(0).astype(int)
    )

In [24]:
total_stops_transformer = FunctionTransformer(func=is_direct_flight)

## Column Transformer

In [25]:
column_transformer = ColumnTransformer(transformers=[
    ('airline_transformer', airline_transformer, ['airline']),
    ('date_transformer', date_tranformer, ['date_of_journey']),
    ('location_union_transformer', location_union_transformer, ['source', 'destination']),
    ('time_union_transformer', time_union_transformer, ['dep_time', 'arrival_time']),
    ('duration_log_transformer', duration_log_transformer, ['duration_minute']),
    ('total_stops_trasformer', total_stops_transformer, ['total_stops'])
])

In [26]:
final_input_df = column_transformer.fit_transform(X_train, y_train)

In [27]:
final_input_columns = final_input_df.columns.to_list()

In [28]:
len(final_input_columns)

21

In [29]:
print(final_input_columns)

['airline_transformer__airline_Air India', 'airline_transformer__airline_Indigo', 'airline_transformer__airline_Jet Airways', 'airline_transformer__airline_Multiple Carriers', 'airline_transformer__airline_other', 'date_transformer__date_of_journey_month', 'date_transformer__date_of_journey_day_of_week', 'date_transformer__date_of_journey_day_of_year', 'location_union_transformer__source', 'location_union_transformer__destination', 'location_union_transformer__source_is_north', 'location_union_transformer__destination_is_north', 'time_union_transformer__dep_time_hour', 'time_union_transformer__dep_time_minute', 'time_union_transformer__arrival_time_hour', 'time_union_transformer__arrival_time_minute', 'time_union_transformer__dep_time_part_of_day', 'time_union_transformer__arrival_time_part_of_day', 'duration_log_transformer__duration_minute', 'total_stops_trasformer__total_stops', 'total_stops_trasformer__is_direct_flight']


## Save the preprocessed CSV file

In [30]:
export_dataset(column_transformer.fit_transform(get_dataset(TRAIN_DATASET_NAME), y_train), TRAIN_PRE_DATASET_NAME)
export_dataset(column_transformer.transform(get_dataset(VAL_DATASET_NAME)), VAL_PRE_DATASET_NAME)
export_dataset(column_transformer.transform(get_dataset(TEST_DATASET_NAME)), TEST_PRE_DATASET_NAME)